Building an algorithm that can discern what datasets have been used by publications

In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")
from wordcloud import WordCloud, STOPWORDS
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [ ]:
path = '../input/coleridgeinitiative-show-us-the-data/train/'
for f in os.listdir(path):
    f = open(path+f)
    
    sample = json.load(f)
    break
    
sample[0]

In [ ]:
for i, s in enumerate(sample):
    print(s['section_title'])
    
    if (i>5):
        break

In [ ]:
train=pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
submit = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
test=submit[['Id']]

train.shape, test.shape

In [ ]:
train.head()

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def get_train_text(filename):
    path = '../input/coleridgeinitiative-show-us-the-data/train/' + filename + '.json'
    df = pd.read_json(path)
    
    text = ' '.join(df['text'])
    return text

def get_test_text(filename):
    path = '../input/coleridgeinitiative-show-us-the-data/test/' + filename + '.json'
    df = pd.read_json(path)
    
    text = ' '.join(df['text'])
    return text

In [ ]:
for i in tqdm(range(len(train))):
    filename = train.loc[i,'Id']
    train.loc[i,'text'] = clean_text(get_train_text(filename))
    
for i in range(len(test)):
    filename = test.loc[i,'Id']
    test.loc[i,'text'] = get_test_text(filename)

In [ ]:
for i in range(len(train)):
    label = train.loc[i,'cleaned_label']
    text = train.loc[i,'text']
    
    if (label in text):
        train.loc[i,'is_present']=1
    else:
        train.loc[i,'is_present']=0

In [ ]:
train['is_present'].value_counts()

In [ ]:
dataset_labels = train['cleaned_label'].unique()

In [ ]:
for i in range(len(test)):
    text = clean_text(test.loc[i,'text'])
    
    prediction_labels=[]
    for label in dataset_labels:
        if (label in text):
            prediction_labels.append(label)
            
    test.loc[i,'PredictionString'] = '|'.join(prediction_labels)

In [ ]:
test

In [ ]:
submit['PredictionString'] = test['PredictionString']
submit

In [ ]:
submit.to_csv('submission.csv',index=False)